<a href="https://colab.research.google.com/github/araldi/FS22---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L/blob/main/Week3/Week3_homework_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Blood biochemical parameters and physical features in diabetic patients versus healthy individuals

Sometimes real data is complicated. In this week's homework, obtaining the final dataset will be more challenging than usual.

You can find below the links to four datasets which are parts of one bigger dataset.

You will also find two dictionaries. Use one to decode the column codes to their content description, the other one to decode the diabetic status of the participants.



Dictionaries:
```
https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/raw/main/Week3/Week3_homework_dictionary_part2.csv


https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/raw/main/Week3/Week3_homework_dictionary_part1.csv
```

Data:
```
https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/blob/main/Week3/Week3_homework_dataA.csv?raw=true


https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/blob/main/Week3/Week3_homework_dataB.csv?raw=true


https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/blob/main/Week3/Week3_homework_dataC.csv?raw=true


https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/blob/main/Week3/Week3_homework_dataD.csv?raw=true
```



#### Tasks


*   Explore the datasets and appropriately merge/concatenate them to obtain one dataset. 

*   Prepare the dataset: remove duplicates, remove NaN and missing diabetes data.

* How many diabetics (in percentage) are in the dataset?  


*   Find the mean and standard deviation of glycated haemoglobin and glucose in healthy patients and in patients with diabetes. 

*  Divide the population by age, and find mean and standard deviation of glycated haemoglobin and glucose in healthy patients and in patients with diabetes in the different age groups (example: younger than 50, 50-54, 55-59, 60-64, 65-69, older than 70)

*   Optional: obtain the same information for males and females separately

## Solution

#### Import the datasets and inspect them

In [ ]:
import pandas as pd

In [ ]:
dict2 = pd.read_csv('https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/raw/main/Week3/Week3_homework_dictionary_part2.csv')

In [ ]:
dict1 = pd.read_csv('https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/raw/main/Week3/Week3_homework_dictionary_part1.csv')

In [ ]:
dict1

In [ ]:
dict2

Dict1 is the decoding of the columns, dict2 is the decoding of the diabetes status (code '2443-0.0')

In [ ]:
dfA = pd.read_csv('https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/blob/main/Week3/Week3_homework_dataA.csv?raw=true')

In [ ]:
dfB = pd.read_csv('https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/blob/main/Week3/Week3_homework_dataB.csv?raw=true')

In [ ]:
dfC = pd.read_csv('https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/blob/main/Week3/Week3_homework_dataC.csv?raw=true')

In [ ]:
dfD = pd.read_csv('https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/blob/main/Week3/Week3_homework_dataD.csv?raw=true')

In [ ]:
dfA.info()

In [ ]:
dfC.info()

In [ ]:
dfB.info()

In [ ]:
dfD.info()

### Prepare the dataset

#### Merge and concatenate the datasets

Looks like there are two pairs of dataframes with different patients and same columns. Let's concatenate the the dfs with same columns (A and C, and B and D)

In [ ]:
df1 = pd.concat([dfB, dfD])

In [ ]:
df1

In [ ]:
df2 = pd.concat([dfA, dfC])

In [ ]:
df2

df1 and df2 have probably duplicate individuals, or some individuals in one dataframe are not present in the other one. Since the info we need is on both dataframes, we need to get rid of individuals for which there is not one or the other info. Let's merge the new dataframes df1 and df2 with the *inner* method.

In [ ]:
df = pd.merge(df1, df2, how='inner', on='eid')

In [ ]:
df

#### Clean the dataset (remove duplicates, rename columns, remove NaN)

There might still be duplicates. Let's get rid of them. And to be clean, let's also reset the index.

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
df

Rename the columns to understand which ones we need (before cleaning the NaN)

In [ ]:
# info on the columns are in dict1
dict1

In [ ]:
type(dict1)

In [ ]:
cols_dict = {}
for index, value in enumerate(dict1['Code']):
  cols_dict[value] = dict1.loc[index, 'Description']
cols_dict

In [ ]:
df = df.rename(columns = cols_dict)
df

In [ ]:
df.isna().sum()

The columns we need are Glucose, Glycated haemoglobin (HbA1c), Gender, Diabetes diagnosed by doctor, AgeRecruit. Let's get rid of NaN in these columns only.

In [ ]:
df = df.dropna(subset= ['Glucose [mmol/L]', 'Glycated haemoglobin (HbA1c) [mmol/mol]', 'Gender', 'Diabetes diagnosed by doctor', 'AgeRecruit'])

In [ ]:
df

Deal with missing diabetes info (coding in dict2)


In [ ]:
dict2

Let's see how many missing values we have

In [ ]:
df['Diabetes diagnosed by doctor'].value_counts()

We will replace the values -1 and -3 with a NaN using NumPy.select() 

In [ ]:
import numpy as np

condition = [(df['Diabetes diagnosed by doctor'] <0)] #this satisfies both the -1 and -3 conditions
choice = [np.nan]
df['Diabetes'] = np.select(condition, choice, default = df['Diabetes diagnosed by doctor'])

In [ ]:
df['Diabetes'].value_counts() # the NaN are not considered!

In [ ]:
df = df.dropna(subset =['Diabetes'])

As an alternative to np.select and dropna, we can subset the dataset for values of 'Diabetes diagnosed by doctor' >= 0 

In [ ]:
df = df[df['Diabetes diagnosed by doctor']>=0]

In [ ]:
df

### How many diabetics in the cohort?

In [ ]:
df['Diabetes'].value_counts(normalize = True)

In [ ]:
percent_diabetics = df['Diabetes'].value_counts(normalize = True).values[1]

In [ ]:
print('There are {:.1%} diabetics in the cohort'.format(percent_diabetics) ) # format as a percentage with 2 decimal digits

### Glucose and glycated haemoglobin in patients with or without diabetes

In [ ]:
diabetes_yes_no = {0: 'patients without diabetes', 1: 'patients with diabetes'}

for i in range(0,2):
  subset = df[df['Diabetes'] == i]
  for biomarker in ['Glucose [mmol/L]', 'Glycated haemoglobin (HbA1c) [mmol/mol]']:
    mean = np.mean(subset[biomarker])
    std = np.std(subset[biomarker])
    print('In {}, {} mean is {:.3f} and standard deviation is {:.3f}'.format(diabetes_yes_no[i], biomarker, mean, std ))

### Same as above, but also divided by males/females

In [ ]:
diabetes_yes_no = {0: 'patients without diabetes', 1: 'patients with diabetes'}
gender_dict = {0 :'Female', 1: 'Male'}

for gender in gender_dict:
  gender_subset = df[df["Gender"] == gender]

  for i in range(0,2):
    subset = gender_subset[gender_subset['Diabetes'] == i]
    for biomarker in ['Glucose [mmol/L]', 'Glycated haemoglobin (HbA1c) [mmol/mol]']:
      mean = np.mean(subset[biomarker])
      std = np.std(subset[biomarker])
      print('In {} {}, {} mean is {:.3f} and standard deviation is {:.3f}'.format(gender_dict[gender], diabetes_yes_no[i], biomarker, mean, std ))

In [ ]:
# let's make it a table

diabetes_yes_no = {0: 'no', 1: 'yes'}
gender_dict = {0 :'Female', 1: 'Male'}

columns = ['Gender', 'Diabetes?',   
           'Glucose [mmol/L] mean', 'Glucose [mmol/L] std',
           'Glycated haemoglobin (HbA1c) [mmol/mol] mean', 'Glycated haemoglobin (HbA1c) [mmol/mol] std']
table_gender_diabetes = pd.DataFrame(columns = columns)

for gender in gender_dict:
  gender_subset = df[df["Gender"] == gender]

  for i in range(0,2):
    subset = gender_subset[gender_subset['Diabetes'] == i]
    mean = {}
    std = {}
    for biomarker in ['Glucose [mmol/L]', 'Glycated haemoglobin (HbA1c) [mmol/mol]']:
      mean[biomarker] = np.mean(subset[biomarker])
      std[biomarker] = np.std(subset[biomarker])
    
    to_add = pd.Series([gender_dict[gender], diabetes_yes_no[i],
                        mean['Glucose [mmol/L]'], std['Glucose [mmol/L]'],
                        mean['Glycated haemoglobin (HbA1c) [mmol/mol]'], std['Glycated haemoglobin (HbA1c) [mmol/mol]'],
                          
                          ], index = columns)
    table_gender_diabetes = table_gender_diabetes.append(to_add, ignore_index=True)
  



In [ ]:
table_gender_diabetes

Determine levels of glucose and HbA1C in different ages

In [ ]:
# First, let's check the age distribution of the population
df['AgeRecruit'].describe()

In [ ]:
conditions = [df['AgeRecruit'] < 50, 
              (df['AgeRecruit'] >= 50) & (df['AgeRecruit'] < 55),
              (df['AgeRecruit'] >= 55) & (df['AgeRecruit'] < 60),
              (df['AgeRecruit'] >= 60) & (df['AgeRecruit'] < 65),
              df['AgeRecruit'] >= 65
              ]

choices = [0, 1, 2, 3, 4]

df['AgeBin'] = np.select(conditions, choices, default = np.nan)

In [ ]:
# let's check that there are enough individuals in each bin
df['AgeBin'].value_counts()

In [ ]:
age_bin_dic = {0 : 'younger than 50',
               1: "between 50 and 54",
               2: "between 55 and 60",
               3: "between 60 and 64",
               4: 'older than 65'}

diabetes_yes_no = {0: 'patients without diabetes', 1: 'patients with diabetes'}

for age in age_bin_dic:
  age_subset = df[df["AgeBin"] == age]

  for i in range(0,2):
    subset = age_subset[age_subset['Diabetes'] == i]
    for biomarker in ['Glucose [mmol/L]', 'Glycated haemoglobin (HbA1c) [mmol/mol]']:
      mean = np.mean(subset[biomarker])
      std = np.std(subset[biomarker])
      print('In {} {}, {} mean is {:.3f} and standard deviation is {:.3f}'.format(diabetes_yes_no[i], age_bin_dic[age],  biomarker, mean, std ))
        

In [ ]:
# let's make it a table

diabetes_yes_no = {0: 'no', 1: 'yes'}

age_bin_dic = {0 : 'younger than 50',
               1: "between 50 and 54",
               2: "between 55 and 60",
               3: "between 60 and 64",
               4: 'older than 65'}

columns = ['Age', 'Diabetes?',   
           'Glucose [mmol/L] mean', 'Glucose [mmol/L] std',
           'Glycated haemoglobin (HbA1c) [mmol/mol] mean', 'Glycated haemoglobin (HbA1c) [mmol/mol] std']
table_age_diabetes = pd.DataFrame(columns = columns)

for age in age_bin_dic:
  age_subset = df[df["AgeBin"] == age]

  for i in range(0,2):
    subset = age_subset[age_subset['Diabetes'] == i]
    mean = {}
    std = {}
    for biomarker in ['Glucose [mmol/L]', 'Glycated haemoglobin (HbA1c) [mmol/mol]']:
      mean[biomarker] = np.mean(subset[biomarker])
      std[biomarker] = np.std(subset[biomarker])
    
    to_add = pd.Series([age_bin_dic[age], diabetes_yes_no[i],
                        mean['Glucose [mmol/L]'], std['Glucose [mmol/L]'],
                        mean['Glycated haemoglobin (HbA1c) [mmol/mol]'], std['Glycated haemoglobin (HbA1c) [mmol/mol]'],
                          
                          ], index = columns)
    table_age_diabetes = table_age_diabetes.append(to_add, ignore_index=True)
  

In [ ]:
table_age_diabetes